In [ ]:
# https://app.tavily.com/sign-in

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(k=5)
search.invoke("판교 근처 이탈리안 레스토랑은 무엇이 있나?")

ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/소나기.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter)
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [15]:
retriever.get_relevant_documents(
    "소녀는 소년과 어디에서 처음 만났나?"
)

[Document(id='1c24994e-4b34-4736-9d3d-05eaa55599ac', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.11131', 'creationdate': '2021-11-29T20:05:50+09:00', 'author': 'MYHOME', 'moddate': '2021-11-29T20:05:50+09:00', 'pdfversion': '1.4', 'source': 'data/소나기.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='소나기황순원소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯이.벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다.소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다.요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.다음 날은 좀 늦게 개울가로 나왔다.이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다.한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 것이리라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사람이 있어야 길을 비킬 모양이다.그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다. 그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다.다 건너가더니만 홱 이리로 돌아서며,“이 바보.”조약돌이 날아왔다.

In [16]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="PDF 문서로부터 정보를 검색한다. 소년과 소녀에 대한 내용은 이 도구를 사용한다.",
)

In [17]:
tools = [search, retriever_tool]

NameError: name 'search' is not defined

In [18]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

/tmp/ipykernel_38073/2407036373.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [19]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/ubuntu/.local/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)


NameError: name 'tools' is not defined

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({"input": "2022년 월드컵 결승전 결과를 알려줘."})
print(f'답변: {response["output"]}')

In [ ]:
response = agent_executor.invoke({"input": "소년과 소녀는 어디에서 처음 만났나?"})
print(f'답변: {response["output"]}')


In [ ]:
response = agent_executor.invoke(
    {"input": "안녕! 나는 판교에 살고 있는 개발자야. 만나서 반가와", "chat_history": []}
)
print(f'답변: {response["output"]}')


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

response = agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕! 나는 판교에 살고 있는 개발자야. 만나서 반가와"),
            AIMessage(
                content="안녕하세요! 판교에서 개발하시는 건가요? 어떤 도움이 필요하신가요? 함께 무엇을 이야기해볼까요?"
            ),
        ],
        "input": "내가 어디 산다고 했지?",
    }
)
print(f'답변: {response["output"]}')


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "소년은 소녀에게 무엇을 주려고 했지?"
    },
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "언제 주었지?"
    },
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")